In [1]:
import sys; sys.path.insert(0, "..")

import aoc

year, day = 2017, 14

puzzle = aoc.setup(year, day)
plines = puzzle.splitlines()

# Day 14

In [2]:
def knot(inp):
    lengths = [*map(ord, inp), 17, 31, 73, 47, 23]
    nums = [*range(256)]
    pos = 0
    skip = 0
    
    def rev(a, b):
        a %= len(nums)
        b = (b - a) % len(nums)
        nums[:] = nums[a:] + nums[:a]
        nums[:b] = nums[:b][::-1]
        nums[:] = nums[-a:] + nums[:-a]
    
    for _ in range(64):
        for length in lengths:
            rev(pos, pos + length)
            pos += length + skip
            skip += 1
    
    dense = []
    for i in range(16):
        x = 0
        for j in range(16): x ^= nums[i*16+j]
        dense.append(x)
    
    return "".join(f"{x:02x}" for x in dense)

### Puzzle 1

In [3]:
def solve1():
    out = 0
    for i in range(128):
        k = bin(int(knot(f"{puzzle}-{i}"), 16))[2:].zfill(128)
        out += k.count("1")
    return out

solve1()

8292

In [4]:
%timeit solve1()

2.09 s ± 177 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Puzzle 2

In [5]:
class UnionFind:
    def __init__(self, n):
        self.parents = list(range(n))
    
    def find(self, x):
        if self.parents[x] == x: return x
        out = self.find(self.parents[x])
        self.parents[x] = out
        return out
        
    def merge(self, a, b):
        a = self.find(a)
        b = self.find(b)
        self.parents[a] = b

def solve2():
    last = None
    uf = UnionFind(128 * 128)
    free = 0
    for i in range(128):
        k = bin(int(knot(f"{puzzle}-{i}"), 16))[2:].zfill(128)
        for j in range(128):
            if k[j] != "1":
                free += 1
                continue
            if i and last[j] == "1": uf.merge((i-1)*128+j, i*128+j)
            if j and k[j-1] == "1": uf.merge(i*128+j-1, i*128+j)
        last = k
        
    groups = set()
    for i in range(128*128): groups.add(uf.find(i))
    return len(groups) - free

solve2()

1069

In [6]:
%timeit solve2()

2.49 s ± 467 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
